In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri May 26 13:01:53 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    22W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 72.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 101.4 MB/s eta 0:00:00


In [ ]:
import os, torch
import pandas as pd
import time
from torch.utils.data import TensorDataset, SequentialSampler, DataLoader, Dataset
from transformers import BertTokenizer, BertModel
import numpy as np
from torch import nn
from tqdm import tqdm
from torch.optim import SGD
from torch.nn.functional import softmax
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import TensorDataset
from transformers import AdamW

In [ ]:
# If there's a GPU available...
if torch.cuda.is_available():

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla V100-SXM2-16GB


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/licenta/')

Mounted at /content/drive


In [ ]:
print(os.listdir('./'))

['get_data_after_bert.py', 'run_prin_bert_freezed.ipynb', 'data', 'cls_output', 'logs', 'first_try.ipynb', 'modele']


In [ ]:
# def loss_fn(outputs, targets):
#     return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [ ]:
def get_all_data_from_filename(full_filename):
    df = pd.read_csv(full_filename, delimiter='\t', header=None, names=['iid','src','native_speaker','original','dest','text','direct','label'])
    return df

In [ ]:
def get_text_and_label(df):
    return df.text.values[1:], df.label.values[1:]

In [ ]:
def import_model(name='bert-base-uncased'):
    token = BertTokenizer.from_pretrained(name)
    mod = BertModel.from_pretrained(name)
    return token, mod

In [ ]:
# class DatasetBERT(Dataset):
#     def __init__(self, X, Y, tokenizer, max_len, device):
#         Y = np.stack(Y).astype(float)
#         self.X = X
#         self.Y = torch.tensor(Y, device=device).float()
#         self.tokenizer = tokenizer
#         self.max_len = max_len

#     def __len__(self):
#         return self.X.shape[0]
    
#     def __getitem__(self, index):
#       text = self.X[index]
#       inputs = self.tokenizer.encode_plus(
#                             text,                      # Sentence to encode.
#                             add_special_tokens = True, # Add '[CLS]' and '[SEP]'
#                             max_length = self.max_len,           # Pad & truncate all sentences.
#                             pad_to_max_length = True,
#                             return_attention_mask = True,   # Construct attn. masks.
#                             return_tensors = 'pt',     # Return pytorch tensors.
#                        )
#       ids = inputs['input_ids']
#       # print(f"class - input ids shape = {ids.shape}")
#       mask = inputs['attention_mask']
#       # print(f"class - input mask shape = {mask.shape}")
#       token_type_ids = inputs["token_type_ids"]
#       # print(f"class - input token type shape = {token_type_ids.shape}")
#       return {
#             'ids': torch.tensor(ids, dtype=torch.long),
#             'mask': torch.tensor(mask, dtype=torch.long),
#             'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
#             'labels': torch.tensor(self.Y[index], dtype=torch.float)
#         }

In [ ]:
k = 6 # CHANGE THIS !!!!!!
class MLP(nn.Module):
  def __init__(self, mod):
    super().__init__()
    self.bert = mod
    self.layer1 = nn.Linear(768, 512)
    self.layer2 = nn.Linear(512, 128)
    self.layer3 = nn.Linear(128, 2)
    self.activation = nn.ReLU()

  def forward(self, ids, mask, token_type_ids):
    out= self.bert(ids, attention_mask = mask, token_type_ids = token_type_ids)['pooler_output']
    out = self.activation(out)
    out = self.activation(self.layer1(out))
    out = self.layer2(out)
    out = self.layer3(out)
    return out

In [ ]:
TOKENIZER, model_bert = import_model()
MAX_LEN = 256
BATCH_SIZE = 32

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# torch.save(model_bert, './danibaciutest')

In [ ]:
NUM_EPOCHS = 5

In [ ]:
def transform_input(tokenizer, X, Y, max_len):
  print(X.shape)
  print(Y.shape)
  print(type(Y[0]))

  input_ids = []
  attention_masks = [] 
  token_type_ids = []

  for sentence in X:

    encoded_dict = tokenizer.encode_plus(
                            sentence,                      # Sentence to encode.
                            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                            max_length = max_len,           # Pad & truncate all sentences.
                            padding = 'max_length',
                            truncation = True,
                            return_attention_mask = True,   # Construct attn. masks.
                            return_token_type_ids=True,
                            return_tensors = 'pt',     # Return pytorch tensors.
                    )
    
    input_ids.append(encoded_dict["input_ids"])
    attention_masks.append(encoded_dict["attention_mask"])
    token_type_ids.append(encoded_dict["token_type_ids"])

  input_ids = torch.cat(input_ids, dim=0)
  attention_masks = torch.cat(attention_masks, dim=0)
  token_type_ids = torch.cat(token_type_ids, dim=0)
  labels = torch.tensor([int(a) for a in Y])

  data = TensorDataset(input_ids, attention_masks, token_type_ids, labels)
  return data


In [ ]:
def get_dataloader_from_filename_batch(filenames, batch_size, tokenizer, max_len, shuffle = True):
  date_train_X, date_train_Y = np.array([]), np.array([])
  for filename in filenames:
    data_x, data_y = get_text_and_label(get_all_data_from_filename(filename))
    date_train_X = np.append(date_train_X, data_x)
    date_train_Y = np.append(date_train_Y, data_y)

  dataset_train = transform_input(tokenizer, date_train_X, date_train_Y, max_len)

  dataloader_train = DataLoader(dataset_train, batch_size, shuffle = shuffle)
  return dataloader_train

In [ ]:
dataloader_train = get_dataloader_from_filename_batch(['./data/mono_en_es_train.tsv', './data/mono_en_es_dev.tsv'], BATCH_SIZE, TOKENIZER, MAX_LEN)
dataloader_validation = get_dataloader_from_filename_batch(['./data/mono_en_es_test.tsv'], BATCH_SIZE, TOKENIZER, MAX_LEN, False)

(35916,)
(35916,)
<class 'str'>
(6344,)
(6344,)
<class 'str'>


In [ ]:
def compute_accuracy(predictions, labels):
    """
    Compute accuracy given the predictions of a binary classifier and the 
    ground truth label.
    predictions: list of model predictions (0 or 1)
    labels: list of ground truth labels (0 or 1)
    """
    num_correct = len([(p,l) for (p,l) in zip(predictions,labels) if p==l])
    epoch_accuracy = num_correct/len(labels)
    
    return epoch_accuracy

In [ ]:
def train_epoch(model, train_dataloader, loss_crt, optimizer, device) :

    model.train()
    epoch_loss = 0.0
    num_batches = len(train_dataloader)
    predictions = []
    labels = []

    for idx, batch in enumerate(train_dataloader):
        batch_ids, batch_attention, batch_token_type_id, batch_labels = batch

        # move data to GPU
        batch_ids = batch_ids.to(device)
        batch_attention = batch_attention.to(device)
        batch_token_type_id = batch_token_type_id.to(device)
        batch_labels = batch_labels.to(device)


        output = model(batch_ids, batch_attention,batch_token_type_id)

        batch_predictions = torch.argmax(output, dim=1)
        
        predictions += batch_predictions.tolist()
        labels += batch_labels.squeeze().tolist()

        loss = loss_crt(output, batch_labels.squeeze())
        loss_scalar = loss.item()

        loss.backward()

        optimizer.step()
        model.zero_grad()

        epoch_loss += loss_scalar

    epoch_loss = epoch_loss/num_batches

    return epoch_loss, predictions, labels


In [ ]:
def eval_epoch(model, val_dataloader, loss_crt, device):
    model.eval()
    epoch_loss = 0.0
    num_batches = len(val_dataloader)
    predictions = []
    labels = []
    with torch.no_grad():
        for idx, batch in enumerate(val_dataloader):
            batch_ids, batch_attention, batch_token_type_id,batch_labels = batch

            # move data to GPU
            batch_ids = batch_ids.to(device)
            batch_attention = batch_attention.to(device)
            batch_token_type_id = batch_token_type_id.to(device)
            batch_labels = batch_labels.to(device)
            
            output = model(batch_ids, batch_attention,batch_token_type_id)

            # batch_size
            batch_predictions = torch.argmax(output, dim=1)
            predictions += batch_predictions.tolist()
            labels += batch_labels.squeeze().tolist()

            loss = loss_crt(output, batch_labels.squeeze())
            loss_scalar = loss.item()

            epoch_loss += loss_scalar

    epoch_loss = epoch_loss/num_batches

    return epoch_loss, predictions, labels


In [ ]:
# os.listdir('./modele/etapa1/')

In [ ]:
def run(model_bert, train_dataloader, test_dataloader, _learning_rate, name):

    model = MLP(model_bert)
    model.to(device)

    loss_fn = torch.nn.CrossEntropyLoss()

    train_losses, val_losses = [], []
    train_accuracies, val_accuracies = [], []
    best_val_acc = 0.0

    optimizer = AdamW(model.parameters(), lr=_learning_rate)

    for i in range(NUM_EPOCHS):

        train_epoch_loss, train_predictions, train_labels = train_epoch(
                                                                          model, 
                                                                          train_dataloader, 
                                                                          loss_fn, 
                                                                          optimizer,
                                                                          device
                                                                      )
        dev_epoch_loss, dev_predictions, dev_labels = eval_epoch(model, test_dataloader, loss_fn, device)
        complet_name = './modele/spa/etapa2/' + name + '_' + str(_learning_rate) + '_' + str(i)
        torch.save(model, complet_name)
        # print(f"off:S-a salvat modelul cu numele {complet_name}")

        dev_acc = compute_accuracy(dev_predictions, dev_labels)
        train_acc = compute_accuracy(train_predictions, train_labels)

        train_losses.append(train_epoch_loss)
        val_losses.append(dev_epoch_loss)
        train_accuracies.append(train_acc)
        val_accuracies.append(dev_acc)

        print("Train loss: ", train_epoch_loss)
        print("Test loss: ", dev_epoch_loss)
        print("Acc train: ", train_acc)
        print("Acc test: ", dev_acc)

    return train_losses, val_losses, train_accuracies, val_accuracies


In [ ]:
import time

for _learning_rate in [4*0.00001, 2*0.00001] :
  print('\n\n-----------------------------------\n\n')
  print(f'learning rate = {_learning_rate}')
  started_at = time.time()
  train_losses, val_losses, train_accuracies, val_accuracies = run(model_bert, dataloader_train, dataloader_validation, _learning_rate, 'model' + str(k))
  completed_at = time.time()

  print(f' For LR={_learning_rate} time elapsed is {int(completed_at - started_at)}')




-----------------------------------


learning rate = 4e-05


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Train loss:  0.37930603516441525
Test loss:  0.21853565437784747
Acc train:  0.8013977057578795
Acc test:  0.9047919293820933
Train loss:  0.2152144507084536
Test loss:  0.20858379560124934
Acc train:  0.8929168058803876
Acc test:  0.9153530895334174
Train loss:  0.14448655018116865
Test loss:  0.2069699600310856
Acc train:  0.9228755986189999
Acc test:  0.9199243379571248
Train loss:  0.11046547292775656
Test loss:  0.2803200532356979
Acc train:  0.936824813453614
Acc test:  0.9134615384615384
Train loss:  0.101795274568477
Test loss:  0.25459946029768066
Acc train:  0.937854994988306
Acc test:  0.914249684741488
 For LR=4e-05 time elapsed is 2455


-----------------------------------


learning rate = 2e-05
Train loss:  0.09872037578081398
Test loss:  0.3245152140255016
Acc train:  0.942449047778149
Acc test:  0.9189785624211854
Train loss:  0.08050389891743386
Test loss:  0.38015991541786337
Acc train:  0.9456509633589486
Acc test:  0.9188209331651954
Train loss:  0.0734559291352396